<a href="https://colab.research.google.com/github/matinmoezzi/ebola-virus-ode-dnn/blob/main/ebola_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
lmbda = 0.6321
mu = 0.9704
beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8, beta9, beta10, beta11 = 0.2877, 0.7613, 0.4389, 0.1234, 0.2431, 0.4000, 0.3000, 0.2345, 0.3312, 0.4123, 0.5111
mu1, mu2, mu3, mu4, mu5, mu6 = 0.0432, 0.2006, 0.0656, 0.9764, 0.6704, 0.0286

In [ ]:
Q=beta1 + beta4 + beta6
R=beta5 + beta7
M=(1/beta1)*(mu1+mu2)
N=lmbda*(beta5 + beta7)
P=beta2
p=mu+beta8
T=beta10*(mu1+mu2)/(beta2*beta9)
U=lmbda*beta10*(beta1+beta4+beta6)
J=beta2*beta9
j=mu + beta8
k=beta10/beta9
A=(beta2*(mu6+beta11)+beta9*(beta3+mu3+mu4)-beta10*(mu1+mu2))*(mu1+mu2)
a=mu+beta8
B=lmbda**2*(beta1+beta4+beta6)*(beta9*(beta5+beta7)-beta10*(beta1+beta4+beta6))
C=lmbda*(-1*beta9*beta2*beta8-beta9*(mu1+mu2)*(beta5+beta7)+(beta1+beta4+beta6)*(2*beta10*(mu1+mu2)-2*beta2*(mu6+beta11)-beta9*(mu6+beta11)))
c=mu+beta8
F=beta2*(beta10*(mu1+mu2)-beta2*(mu6+beta11)-beta9*(beta3+mu3+mu4))
f=mu+beta8
G=lmbda*beta2*(beta9*(beta5+beta7)-beta10*(beta1+beta4+beta6))
g=mu+beta8

In [ ]:
print(f"Q: {round(Q,4)}")
print(f"R: {round(R,4)}")
print(f"M: {round(M,4)}")
print(f"N: {round(N,4)}")
print(f"P: {round(P,4)}")
print(f"p: {round(p,4)}")
print(f"T: {round(T,4)}")
print(f"U: {round(U,4)}")
print(f"J: {round(J,4)}")
print(f"j: {round(j,4)}")
print(f"k: {round(k,4)}")
print(f"A: {round(A,4)}")
print(f"a: {round(a,4)}")
print(f"B: {round(B,4)}")
print(f"C: {round(C,4)}")
print(f"c: {round(c,4)}")
print(f"F: {round(F,4)}")
print(f"f: {round(f,4)}")
print(f"G: {round(G,4)}")
print(f"g: {round(g,4)}")
print(f"QM: {round(Q*M,4)}")
print(f"QN: {round(Q*N,4)}")
print(f"QT: {round(Q*T,4)}")
print(f"QU: {round(Q*U,4)}")
print(f"R-Qk: {round(R-Q*k,4)}")

In [ ]:
from scipy import optimize

In [ ]:
def func(x):
  return x-Q*M + Q*N/(P*(x+p)) + Q*T - Q*U/(J*(x+j)) - (R - Q * k)*(A * (x + a)**2 + B + C * (x + c)) / (F * (x + f)**2 + G*(x + g))

In [ ]:
def fprime(x):
  return 1 - Q*N/(P*(x+p)**2) + Q*U/(J*(x+j)**2) - (R-Q*k)*((2*A*(x+a)+C)*(F*(x+f)**2+G*(x+g))+((2*F*(x+f)+G)*(A*(x+a)**2+B+C*(x+c))))/(F*(x+f)**2+G*(x+g))**2

In [ ]:
sol=optimize.root_scalar(func,fprime=fprime, x0=0, method='newton')
print(sol.root)

In [ ]:
!pip install neurodiffeq

In [ ]:
from neurodiffeq import diff
from neurodiffeq.conditions import IVP
from neurodiffeq.monitors import Monitor1D
from neurodiffeq.solvers import Solver1D
from neurodiffeq.callbacks import MonitorCallback
from neurodiffeq.networks import FCNN
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import torch

In [ ]:
def system_ode(u1, u2, t): return [diff(u1, t) - torch.cos(t) - u1**2 - u2 + (1 + t**2 + torch.sin(t)**2),
                                   diff(u2, t) - 2*t + (1 + t**2)*torch.sin(t) - u1*u2]

In [ ]:
# specify the initial conditions
init_vals_pc = [
    IVP(t_0=0.0, u_0=0.0),
    IVP(t_0=0.0, u_0=1.0)
]

In [ ]:
monitor = Monitor1D(t_min=0, t_max=3.0, check_every=100)
monitor_callback = MonitorCallback(monitor)

nets_lv = [
    FCNN(n_input_units=1, n_output_units=1,
         n_hidden_units=10, actv=nn.Sigmoid),
    FCNN(n_input_units=1, n_output_units=1, n_hidden_units=10, actv=nn.Sigmoid)
]

# solve the ODE system
solver = Solver1D(
    ode_system=system_ode, conditions=init_vals_pc, t_min=0.0, t_max=3.0, nets=nets_lv
)

In [ ]:
solver.fit(max_epochs=10000, callbacks=[monitor_callback])
solution_pc = solver.get_solution()

In [ ]:
ts = np.linspace(0, 3.0, 300)
u1_net, u2_net = solution_pc(ts, to_numpy=True)
u1_ana, u2_ana = np.sin(ts), 1 + ts**2

plt.figure()
plt.plot(ts, u1_net, label='ANN-based solution of $u_1$')
plt.plot(ts, u1_ana, label='Analytical solution of $u_1$')
plt.plot(ts, u2_net, label='ANN-based solution of $u_2$')
plt.plot(ts, u2_ana, label='Analytical solution of $u_2$')
plt.ylabel('u')
plt.xlabel('t')
plt.title('comparing solutions')
plt.legend()
plt.show()